In [1]:
import os
import requests
import re
from bs4 import BeautifulSoup
import openpyxl

In [2]:
def hasdigit(url):
    return any(char.isdigit() for char in url)

In [3]:
def HavaAlphabet(url):
    return any(char.isalpha() for char in url)

In [4]:
def cnn_business_Filter(url, flag):
    feature1 = url[flag:flag+4]
    if HavaAlphabet(feature1):
        return False
    else:
        return True

In [5]:
cnn_business = 'https://edition.cnn.com/business'

In [6]:
saving_path = '.\\7_websites_output'
if not os.path.exists(saving_path):
    os.makedirs(saving_path)

In [7]:
CNN_homePage = ".zn__containers"
CNN_NewsPage = ".zn-body__paragraph"

In [8]:
def ft_CNN():
    # Crawler ----------------------------------------------------------------------
    prefix = 'https://edition.cnn.com'
    r = requests.get(cnn_business)

    if r.status_code == 200:  # print(r.status_code)
        soup = BeautifulSoup(r.text, 'html.parser')
        result = soup.select(CNN_homePage)

    raw_url_list = []
    target_url_list = []
    for results in result:
        class_a = results.find_all("a")

        for i in class_a:
            if i['href'][0] == '/':
                raw_url_list.append(str(prefix+i['href']))
            else:
                raw_url_list.append(str(i['href']))

    for url in raw_url_list:
        if cnn_business_Filter(url, len(prefix)+1):
            if url in target_url_list:
                pass
            else:
                target_url_list.append(url)

    # Save ----------------------------------------------------------------------
    workbook = openpyxl.Workbook()
    sheet = workbook.active

    ctr = 1

    for url in target_url_list[:]:
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            result = soup.select(CNN_NewsPage)
            title_select = soup.find("h1")
            sheet['A'+str(ctr)] = title_select.text
            sheet['B'+str(ctr)] = url
            # print("-----------------------------------------------------------------------------------------------")
            # print(title_select.text)
            # print(url)


            content = []
            for results in result:
                content.append(results.text)

            if content == []:
                print("Can't crawler the web")
                print(url)
                sheet['C'+str(ctr)] = "Can't Crawler the Web"
                sheet['D'+str(ctr)] = ''
                ctr += 1
            else:
                for sentence in content:
                    sheet['D'+str(ctr)] = str(sentence)
                    ctr += 1

    workbook.save(saving_path+'\\CNN.xlsx')
    
    print('finish')

In [9]:
ft_CNN()

finish
